In [15]:
import pandas as pd
import re
data = pd.read_csv('pop_docs_nlp-1.csv', index_col=0)
test = re.sub(r'\\n',r'', (data.text[1]))

## Attempt 1
Custom pipeline

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

pretrained_model = "jamarju/roberta-large-bne-squad-2.0-es"
tokenizer = AutoTokenizer.from_pretrained(pretrained_model)
model = AutoModelForQuestionAnswering.from_pretrained(pretrained_model)

In [19]:
context = test
question = '¿Cuáles son los objetivos de la Unesco?'

In [28]:
inputs = (tokenizer(question, context, return_tensors='pt', padding='longest', truncation='only_second'))

In [29]:
outputs = model(**inputs)
start_logits = outputs.start_logits
end_logits = outputs.end_logits
print(start_logits.shape, end_logits.shape)

torch.Size([1, 512]) torch.Size([1, 512])


In [31]:
sequence_ids = inputs.sequence_ids()

# Mask out-of-context tokens
mask = [i != 1 for i in sequence_ids]

# Unmask the [CLS] token
mask[0] = False
mask = torch.tensor(mask)[None]

# Neg large nr's exp will be zero
start_logits[mask] = -10000
end_logits[mask] = -10000

# Convert logits to probabilities
start_probs = torch.nn.functional.softmax(start_logits, dim=-1)[0]
end_probs = torch.nn.functional.softmax(end_logits, dim=-1)[0]

In [34]:
scores = start_probs[:, None] * end_probs[None, :]
scores = torch.triu(scores)

max_index = scores.argmax().item()
start_index = max_index // scores.shape[1]
end_index = max_index % scores.shape[1]
score = scores[start_index, end_index].item()

In [35]:
inputs_with_offsets = tokenizer(question, context, return_offsets_mapping=True)
offsets = inputs_with_offsets['offset_mapping']

start_char = offsets[start_index]
_, end_char = offsets[end_index]
answer = context[start_char:end_char]

print(f'Answer: "{answer}", Score: "{score:.4f}"') #Indices are off. FIND BUG

TypeError: slice indices must be integers or None or have an __index__ method

#### Attempt 1.2
Use function from [article](https://medium.com/mlwhiz/understanding-bert-with-huggingface-64378237aa8)

In [ ]:
inputs = tokenizer.encode_plus(question, test, add_special_tokens=True, return_tensors="pt")
input_ids = inputs["input_ids"].tolist()[0]
text_tokens = tokenizer.convert_ids_to_tokens(input_ids)

pred = model(**inputs)
answer_start_scores, answer_end_scores = pred['start_logits'][0] ,pred['end_logits'][0] 
answer_start = torch.argmax(answer_start_scores)# Get most likely beginning of answer with the argmax of the score
answer_end = torch.argmax(answer_end_scores) + 1  # Get most likely end of answer with the argmax of the score
answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))
print(f"Question: {question}, Answer: {answer}")

## Attempt 2: Pipeline

In [42]:
from transformers import Pipeline, pipeline
from typing import Dict

In [56]:
pretrained_model ="PremalMatalia/roberta-base-best-squad2"

qa = pipeline(task='question-answering', model=pretrained_model)

def answer_question(pipeline: Pipeline, question: str, paragraph: str) -> Dict:
    input = {"question": question, "context": paragraph}
    return pipeline(input)

In [48]:
qa = pipeline(task='question-answering', model=pretrained_model)

In [49]:
answer_test = answer_question(pipeline=qa, question=, paragraph=context)

/opt/anaconda3/envs/deep/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:707: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)
/opt/anaconda3/envs/deep/lib/python3.8/site-packages/transformers/pipelines/question_answering.py:296: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  p_mask = np.asarray(


In [64]:
question1 = '¿Qué es una respuesta alternativa?'
#'answer': 'objeción de conciencia es una alternativa a la fuerza y a la violencia'}
question2 = '¿Qué constituye la objecion de conciencia?'
#'answer': 'objeción de conciencia es una alternativa a la fuerza y a la violencia'}
#true_answer = 'un mecanismo coherente y eficaz para la defensa de las libertades individuales'
question3 = '¿Existen actos delictuales?'
#'answer': 'actos delictuales contra símbolos y templos'
#true_answer = si
question4 = '¿Qué se debe considerar?'
#'answer': 'discurso de odio'
#true_answer = 'las creencias religiosas son convicciones de propósitos de vida respetables'

Without '¿'

In [70]:
question1 = 'Qué es una respuesta alternativa?'
#'answer': 'objeción de conciencia es una alternativa a la fuerza y a la violencia'}
question2 = 'Qué constituye la objecion de conciencia?'
#'answer': 'una alternativa a la fuerza y a la violencia'
question3 = 'Existen actos delictuales?'
#'answer': 'símbolos y templos'}
#true_answer = 'Si'
question4 = 'Qué se debe considerar?'
#'answer': 'todas las personas'}

Uncased

In [76]:
question1 = '¿qué es una respuesta alternativa?'
#'answer': 'Asociación Internacional para la Libertad Religiosa'} .81 NOT
question2 = '¿qué constituye la objecion de conciencia?'
#'answer': 'una alternativa a la fuerza y a la violencia'} .91
question3 = '¿existen actos delictuales?'
#'answer': 'actos delictuales contra símbolos y templos'} .72
question4 = '¿qué se debe consagrar?'
#'answer': 'derechos fundamentales'} .34

Uncased without '¿'

In [81]:
question1 = 'qué es una respuesta alternativa?'
#'answer': 'objeción de conciencia es una alternativa a la fuerza y a la violencia'} .89
question2 = 'qué constituye la objecion de conciencia?'
#'answer': 'una alternativa a la fuerza y a la violencia'
#true_answer = 'un mecanismo coherente y eficaz para la defensa de las libertades individuales'
question3 = 'existen actos delictuales?'
#'answer': 'símbolos y templos'} .57
question4 = 'qué se debe consagrar?'
#'answer': 'derechos' .44

In [83]:
type(qa)

transformers.pipelines.question_answering.QuestionAnsweringPipeline

#### jamarju model
roBERTa is too slow

In [84]:
pretrained_model1 = "jamarju/roberta-large-bne-squad-2.0-es"
qa = pipeline(task='question-answering', model=pretrained_model1)

In [91]:
question1 = 'Qué es una respuesta alternativa?'
#'answer': 'objeción de conciencia es una alternativa a la fuerza y a la violencia'} .91
question2 = 'Qué constituye la objecion de conciencia?'
#'answer': 'una alternativa a la fuerza y a la violencia'} .97
question3 = 'Existen actos delictuales?'
#'answer': 'símbolos y templos' .58
#true_answer = si
question4 = 'Qué se debe considerar?'
#'answer': 'todas las personas' .45
#true_answer = 'las creencias religiosas son convicciones de propósitos de vida respetables'
answers_jam = ([answer_question(pipeline=qa, question=q, paragraph=context)
                for q in [question1, question2, question3, question4]])

### Lighter Models

In [95]:
bert = "mrm8488/bert-base-spanish-wwm-cased-finetuned-spa-squad2-es"
distilbert = "mrm8488/distill-bert-base-spanish-wwm-cased-finetuned-spa-squad2-es"

In [97]:
qa = pipeline(task='question-answering', model=bert)
answers_bert_noq = ([answer_question(pipeline=qa, question=q, paragraph=context)
                     for q in [question1, question2, question3, question4]])
[print(i) for i in answers_bert_noq]

{'score': 0.3406215012073517, 'start': 1488, 'end': 1523, 'answer': 'derecho a la objeción de conciencia'}
{'score': 0.5367205142974854, 'start': 1527, 'end': 1571, 'answer': 'una alternativa a la fuerza y a la violencia'}
{'score': 0.5960468053817749, 'start': 773, 'end': 791, 'answer': 'símbolos y templos'}
{'score': 0.24172155559062958, 'start': 4039, 'end': 4057, 'answer': 'todas las personas'}


[None, None, None, None]

In [99]:
qa = pipeline(task='question-answering', model=distilbert)
answers_bert_noq = ([answer_question(pipeline=qa, question=q, paragraph=context)
                     for q in [question1, question2, question3, question4]])
[print(i) for i in answers_bert_noq]

{'score': 0.5658420324325562, 'start': 1488, 'end': 1523, 'answer': 'derecho a la objeción de conciencia'}
{'score': 0.6642604470252991, 'start': 1527, 'end': 1571, 'answer': 'una alternativa a la fuerza y a la violencia'}
{'score': 0.5605242848396301, 'start': 773, 'end': 791, 'answer': 'símbolos y templos'}
{'score': 0.5529854893684387, 'start': 4039, 'end': 4057, 'answer': 'todas las personas'}


[None, None, None, None]